In [116]:
from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F

import torch
from torchvision import transforms

import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [117]:
mat_path = "/Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/img/cars_annos.mat"
car_train_path = "/Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/img/cars_train/cars_train"
car_test_path = "/Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/img/cars_test/cars_test"
# Load the .mat file
mat = loadmat(mat_path)

# Access data
# Assuming 'annotations' is a key in the .mat file
annotations = mat['annotations']

# Do something with the data
# For example, print the first item in the annotations
print(annotations[0])


[(array(['car_ims/000001.jpg'], dtype='<U18'), array([[112]], dtype=uint8), array([[7]], dtype=uint8), array([[853]], dtype=uint16), array([[717]], dtype=uint16), array([[1]], dtype=uint8), array([[0]], dtype=uint8))
 (array(['car_ims/000002.jpg'], dtype='<U18'), array([[48]], dtype=uint8), array([[24]], dtype=uint8), array([[441]], dtype=uint16), array([[202]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8))
 (array(['car_ims/000003.jpg'], dtype='<U18'), array([[7]], dtype=uint8), array([[4]], dtype=uint8), array([[277]], dtype=uint16), array([[180]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8))
 ...
 (array(['car_ims/016183.jpg'], dtype='<U18'), array([[25]], dtype=uint8), array([[32]], dtype=uint8), array([[587]], dtype=uint16), array([[359]], dtype=uint16), array([[196]], dtype=uint8), array([[1]], dtype=uint8))
 (array(['car_ims/016184.jpg'], dtype='<U18'), array([[56]], dtype=uint8), array([[60]], dtype=uint8), array([[208]], dtype=uint8

In [118]:
print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'annotations', 'class_names'])


In [119]:
# Extract and print the first few annotations and class names to understand their structure

# Annotations
annotations_sample = mat['annotations'][:5]

# Class names
class_names_sample = mat['class_names'][:, :5]

# Since the structure of annotations and class_names might be complex, let's just print out their shapes or descriptions first
annotations_sample_description = {
    "Type": type(annotations_sample),
    "Shape": annotations_sample.shape,
    "First Element": annotations_sample[0]
}

class_names_sample_description = {
    "Type": type(class_names_sample),
    "Shape": class_names_sample.shape,
    "First Few Names": class_names_sample[:, :5]
}

annotations_sample_description, class_names_sample_description


({'Type': numpy.ndarray,
  'Shape': (1, 16185),
  'First Element': array([(array(['car_ims/000001.jpg'], dtype='<U18'), array([[112]], dtype=uint8), array([[7]], dtype=uint8), array([[853]], dtype=uint16), array([[717]], dtype=uint16), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
         (array(['car_ims/000002.jpg'], dtype='<U18'), array([[48]], dtype=uint8), array([[24]], dtype=uint8), array([[441]], dtype=uint16), array([[202]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
         (array(['car_ims/000003.jpg'], dtype='<U18'), array([[7]], dtype=uint8), array([[4]], dtype=uint8), array([[277]], dtype=uint16), array([[180]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
         ...,
         (array(['car_ims/016183.jpg'], dtype='<U18'), array([[25]], dtype=uint8), array([[32]], dtype=uint8), array([[587]], dtype=uint16), array([[359]], dtype=uint16), array([[196]], dtype=uint8), array([[1]], dtype=uint8)),
         (array(['c

In [120]:
from PIL import Image

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height


In [121]:
import scipy.io
import os

def convert_mat_to_yolo(mat_file_path, output_dir):
    # Load .mat file
    mat = scipy.io.loadmat(mat_file_path)
    annotations = mat['annotations']  # This might vary depending on your .mat file structure
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for annotation in annotations:
        filename = annotation['filename'][0]
        bbox_x1 = annotation['bbox_x1'][0]
        bbox_y1 = annotation['bbox_y1'][0]
        bbox_x2 = annotation['bbox_x2'][0]
        bbox_y2 = annotation['bbox_y2'][0]
        class_id = 0  # Assuming you have only one class, cars. Adjust if you have multiple classes.

        # Convert bbox coordinates to YOLO format (x_center, y_center, width, height)
        x_center = ((bbox_x2 + bbox_x1) / 2) / width  # You need to have the image width
        y_center = ((bbox_y2 + bbox_y1) / 2) / height  # You need to have the image height
        width = (bbox_x2 - bbox_x1) / width
        height = (bbox_y2 - bbox_y1) / height

        # Write to a new text file for each image
        with open(os.path.join(output_dir, filename.replace('.jpg', '.txt')), 'w') as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


In [122]:
# Load the .mat file
mat = loadmat(mat_path)

# Assuming the data structure is a dictionary with keys corresponding to variable names in MATLAB,
# and you are interested in a specific key for the data. Let's say the key is 'annotations':
# Note: Adjust the 'annotations' key according to the actual structure of your .mat file

# Extract the data under 'annotations' or the relevant key
annotations = mat['annotations']  # Adjust 'annotations' as necessary

# Convert the annotations to a list of dictionaries for easier DataFrame conversion
# This step depends on the structure of 'annotations'. Assuming it's a structured array:
data = []
for annotation in annotations[0]:
    data.append({
        'fname': annotation[0][0],  # Adjust field names as necessary
        'bbox_x1': annotation[1][0][0],
        'bbox_y1': annotation[2][0][0],
        'bbox_x2': annotation[3][0][0],
        'bbox_y2': annotation[4][0][0],
        'class': annotation[5][0][0],
        'test': annotation[6][0][0]  # This field might represent whether the image is for testing or not
    })

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = '/Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/data/cars_annotations.csv'
df.to_csv(csv_file_path, index=False)

print(f'Data saved to {csv_file_path}')


Data saved to /Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/data/cars_annotations.csv


In [123]:
df

,fname,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test
0,car_ims/000001.jpg,112,7,853,717,1,0
1,car_ims/000002.jpg,48,24,441,202,1,0
2,car_ims/000003.jpg,7,4,277,180,1,0
3,car_ims/000004.jpg,33,50,197,150,1,0
4,car_ims/000005.jpg,5,8,83,58,1,0
...,...,...,...,...,...,...,...
16180,car_ims/016181.jpg,38,36,375,234,196,1
16181,car_ims/016182.jpg,29,34,235,164,196,1
16182,car_ims/016183.jpg,25,32,587,359,196,1
16183,car_ims/016184.jpg,56,60,208,186,196,1


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   fname    16185 non-null  object
 1   bbox_x1  16185 non-null  uint16
 2   bbox_y1  16185 non-null  uint16
 3   bbox_x2  16185 non-null  uint16
 4   bbox_y2  16185 non-null  uint16
 5   class    16185 non-null  uint8 
 6   test     16185 non-null  uint8 
dtypes: object(1), uint16(4), uint8(2)
memory usage: 284.6+ KB


In [143]:
df2 = df.copy()

In [144]:
# Define a function to adjust the filename
def adjust_filename(fname):
    # Remove the directory path
    filename = fname.split('/')[-1]
    # Check if the filename starts with a zero and remove the first zero
    if filename.startswith('0'):
        filename = filename[1:]
    return filename

# Apply the function to adjust the fname column
df2['fname'] = df2['fname'].apply(adjust_filename)

# Change the fname column type to object
df2['fname'] = df2['fname'].astype('object')


In [145]:
# Assuming df is your DataFrame and contains a column 'fname' with filenames
index = list(set(df2['fname']))
image = random.choice(index)
image_path = f'/Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/img/cars_train/cars_train/{image}.jpg'  # Update the path accordingly

print("Image ID:", image)
print("Attempting to load:", image_path)

# Check if the file exists
if os.path.exists(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        print("Image shape:", img.shape)
    else:
        print("Failed to load image. cv2.imread() returned None.")
else:
    print("File does not exist at the specified path:", image_path)

Image ID: 06404.jpg
Attempting to load: /Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/img/cars_train/cars_train/06404.jpg.jpg
File does not exist at the specified path: /Users/ahmedalmaqbali/Desktop/Muaeen/Rihal/intern/repo/computerVision/img/cars_train/cars_train/06404.jpg.jpg
